In [32]:
from paddlenlp import Taskflow
import pandas as pd

In [33]:
# 先根据cnSchema确定需要获取的关系
schema = {'歌曲名称': ['歌手', '所属专辑', '作词', '作曲']}
ie = Taskflow('information_extraction', schema=schema)

[2024-08-07 02:49:42,966] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'C:\Users\gzfch\.paddlenlp\taskflow\information_extraction\uie-base'.


In [36]:
# 获取数据
df = pd.read_csv("song_infos.csv")
new_names = df['歌曲名称'].to_list()
old_names = pd.read_csv("song_relations.csv", index_col=0).iloc[:,0].to_list()
add_names = list(set(new_names)-set(old_names))
add_infos = []
for name in add_names:
    add_infos.append(df.query("歌曲名称 == @name")["歌曲信息"].iloc[0])

In [38]:
add_names, add_infos

(['新年到',
  '一千年后记得我',
  '爱情Yogurt',
  '加油！',
  '序曲：中场休息',
  '江南',
  '我们很好',
  '对的时间点',
  '序曲：Welcome to the Livehouse',
  'Love U U',
  '想见你想见你想见你',
  'Baby Baby',
  'Cries in a Distance',
  'Despacito 缓缓',
  '序曲：调音',
  'Until the Day',
  '曹操',
  'Castle in the air',
  '一开始',
  '我还年轻 我还年轻',
  'SIXOLOGY',
  '绝不绝'],
 ['由众歌手首次大合唱演绎的《新年到》, 演绎歌手有JJ林俊杰，Rynn林宇中，李志清，BY2， Wil黄威尔， Phoebe菲比，黄启铭，汤小康，曾若冰， CHIN ，满江浤，陈泽耀。 [1]',
  '歌曲《一千年后记得我》，由林俊杰演唱，林秋离填词，林俊杰谱曲，收录于林俊杰2013年3月13日正式发行的第十张个人专辑《因你而在》中的隐藏曲目 [2]',
  '《爱情yogurt》是林俊杰演唱的歌曲，由林怡凤作词，林俊杰作曲，收录在林俊杰2006年2月17日发行的专辑《曹操》中。',
  '《加油!》是林俊杰的音乐作品，林俊杰作曲，收录在《他是...JJ林俊杰》专辑中。',
  '《序曲：中场休息》是林俊杰创作的音乐，收录在林俊杰于2015年12月25日发行的实验专辑《和自己对话》中 [1]。',
  '《第二天堂》是林俊杰于2004年6月4日发行的第二张个人创作专辑，共收录15首歌曲。专辑由许环良监作，林俊杰包办整张专辑作曲，同时参与了编曲及制作。歌词大部分的填写及概念部分，则由林秋离把握。 [1]',
  '《我们很好》是林俊杰于2019年6月13日发布的单曲。该歌曲由葛大为填词，林俊杰谱曲，为电影《少年的你》的主题曲 [1-2]。',
  '《对的时间点》是林俊杰于2019年8月26日发行的专辑，共收录2首歌曲。',
  '《序曲：Welcome to the Livehouse》是林俊杰讲述的口白，收录在林俊杰于2015年12月25日发行的实验专辑《和自己对话》中 [1]。',
  '《Love you you》是林俊杰演唱的一首歌

In [39]:
# 使用paddle进行关系提取
results = []

for i in range(0, len(add_names)):
    s = "["+str(i + 1) + "/"+str(len(add_names))+"]"

    name = add_names[i]
    info = add_infos[i]

    result = ie(info)
    try:
        result = result[0]
        result = result['歌曲名称'][0]
        song_name = result['text']
        relations = result['relations']
    except:
        print(s + name + ":获取关系失败")

    if relations != []:
        print(s + song_name)
        for r_name in ['歌手', '所属专辑', '作词', '作曲']:
            try:
                items = relations[r_name]
                for item in items:
                    results.append([song_name, r_name, item['text']])
            except:
                print(r_name + "关系提取失败")
        print("提取结束")   

c:\Users\gzfch\miniconda3\Lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2478: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  warnings.warn(
c:\Users\gzfch\miniconda3\Lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:1878: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


[1/22]新年到
所属专辑关系提取失败
作词关系提取失败
作曲关系提取失败
提取结束
[2/22]一千年后记得我
提取结束
[3/22]爱情yogurt
提取结束
[4/22]加油!
作词关系提取失败
提取结束
[5/22]序曲：中场休息:获取关系失败
[5/22]加油!
作词关系提取失败
提取结束
[6/22]第二天堂
所属专辑关系提取失败
提取结束
[7/22]我们很好
所属专辑关系提取失败
提取结束
[8/22]对的时间点:获取关系失败
[8/22]我们很好
所属专辑关系提取失败
提取结束
[9/22]序曲：Welcome to the Livehouse:获取关系失败
[9/22]我们很好
所属专辑关系提取失败
提取结束
[10/22]Love you you
所属专辑关系提取失败
提取结束
[11/22]想见你
所属专辑关系提取失败
作词关系提取失败
提取结束
[12/22]baby baby
提取结束
[13/22]Cries In A Distance
提取结束
[14/22]缓缓
所属专辑关系提取失败
提取结束
[15/22]序曲：调音:获取关系失败
[15/22]缓缓
所属专辑关系提取失败
提取结束
[16/22]Until The Day
提取结束
[17/22]曹操:获取关系失败
[17/22]Until The Day
提取结束
[18/22]Castle in the Air
提取结束
[19/22]一开始...
提取结束
[20/22]我还年轻我还年轻
所属专辑关系提取失败
作词关系提取失败
提取结束
[21/22]Sixology
提取结束
[22/22]绝不绝
所属专辑关系提取失败
作词关系提取失败
作曲关系提取失败
提取结束


In [40]:
results

[['新年到', '歌手', '林俊杰'],
 ['一千年后记得我', '歌手', '林俊杰'],
 ['一千年后记得我', '所属专辑', '因你而在'],
 ['一千年后记得我', '作词', '林秋离'],
 ['一千年后记得我', '作曲', '林俊杰'],
 ['爱情yogurt', '歌手', '林俊杰'],
 ['爱情yogurt', '所属专辑', '曹操'],
 ['爱情yogurt', '作词', '林怡凤'],
 ['爱情yogurt', '作曲', '林俊杰'],
 ['加油!', '歌手', '林俊杰'],
 ['加油!', '所属专辑', '他是...JJ林俊杰'],
 ['加油!', '作曲', '林俊杰'],
 ['加油!', '歌手', '林俊杰'],
 ['加油!', '所属专辑', '他是...JJ林俊杰'],
 ['加油!', '作曲', '林俊杰'],
 ['第二天堂', '歌手', '林俊杰'],
 ['第二天堂', '作词', '林秋离'],
 ['第二天堂', '作曲', '林俊杰'],
 ['我们很好', '歌手', '林俊杰'],
 ['我们很好', '作词', '葛大为'],
 ['我们很好', '作曲', '林俊杰'],
 ['我们很好', '歌手', '林俊杰'],
 ['我们很好', '作词', '葛大为'],
 ['我们很好', '作曲', '林俊杰'],
 ['我们很好', '歌手', '林俊杰'],
 ['我们很好', '作词', '葛大为'],
 ['我们很好', '作曲', '林俊杰'],
 ['Love you you', '歌手', '林俊杰'],
 ['Love you you', '作词', '林天爱'],
 ['Love you you', '作曲', '林俊杰'],
 ['想见你', '歌手', '林俊杰'],
 ['想见你', '作曲', '林俊杰'],
 ['baby baby', '歌手', '林俊杰'],
 ['baby baby', '所属专辑', '西界'],
 ['baby baby', '作词', '林秋离'],
 ['baby baby', '作词', '林宇中'],
 ['baby baby', '作曲', '林俊杰'],
 ['Cries In A Distanc

In [43]:
# 保存为csv文件
add_df = pd.DataFrame(columns = ['', '', ''], data = results)
old_df = pd.read_csv("song_relations.csv", index_col=0)
old_df.columns = ['','','']
new_df = pd.concat([old_df, add_df], ignore_index=True)
new_df.to_csv('song_relations.csv')

In [42]:
new_df

,,,
0,The Show,歌手,林俊杰
1,The Show,歌手,Steve Aoki
2,The Show,作词,Steve Aoki
3,The Show,作词,Carl Ryden
4,The Show,作词,James Daniel Lewis
...,...,...,...
1258,Sixology,歌手,林俊杰
1259,Sixology,所属专辑,JJ陆
1260,Sixology,作词,林俊杰
1261,Sixology,作曲,林俊杰
